In [48]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision import datasets, transforms

import numpy as np

In [52]:
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transforms.ToTensor())
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transforms.ToTensor())

In [56]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [177]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.class_size = 10
        self.conv_layer_1 =  nn.Conv2d(1, 14, kernel_size=11, stride=2, padding=1, dilation=1)
        #self.pool = nn.MaxPool2d(2, 2)
        # self.conv_layer_2 = nn.Conv2d(16, 10, (3, 3), 3, 3)
        self.conv_layer_2 = nn.Conv2d(14, 10, kernel_size=11, stride=2, padding=1, dilation=1)
        self.conv_layer_3 = nn.Conv2d(10, 10, kernel_size=1)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        #print(f'{result_1.shape}: {result_1}')
        result_2 = F.relu(self.conv_layer_2(result_1))
        result_3 = F.relu(self.conv_layer_3(result_2))
        #return F.softmax(result_3, dim=1).squeeze(2).squeeze(2)
        return F.log_softmax(result_3, dim=1).squeeze(2).squeeze(2)
    

In [178]:
device = 0

model = Model().to(device)

net = Net().to(device)

In [179]:

# model.forward(image.view(28, 28, 1, 1))
#a = model.forward(image.view(-1))
a = model(train_dataset[0][0].view(1, 1, 28, 28).to(device))
#pixels = np.asarray(train_dataset[0][0])


In [180]:
a.shape

torch.Size([1, 10])

In [181]:
a.squeeze(2).squeeze(2).shape

RuntimeError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(f'CICCIO {output.shape}, {target.shape}')
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    use_cuda = 0
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=100, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
])),
        batch_size=10, shuffle=True, **kwargs)


    model = Model().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

    for epoch in range(1, 10 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308027
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.895597
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.562620
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.623725
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.579683
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.539498
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.721892
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.558714
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.391985
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.548808
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.457493
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.514758

Test set: Average loss: 0.5402, Accuracy: 8807/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.464458
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.526111
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.358293
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.348774
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.418161
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.669457
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.469875
Tra

In [ ]:
    l = datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))]))

In [63]:
l

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ../data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.3081,))
                         )
    Target Transforms (if any): None

In [66]:
l[0][0].shape

torch.Size([1, 28, 28])